In [1]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2 as cv
from torchvision import transforms
import numpy as np
from constants import IMAGE_SIZE

Image.MAX_IMAGE_PIXELS = None

In [2]:
PREFIX = "airs_"
ROOT = "AIRS"
SAVE_ROOT = "Master Dataset"
IMAGES = "Images"
LABELS = "Labels"

Image dimensions -> 10000 x 10000

Split (Train / Val / Test) -> 857 / 94 / 95

In [4]:
img_folder = "image"
label_folder = "label"

for folder in os.listdir(ROOT):
    if ".txt" in folder:
        continue
        
    print(folder)
    ctr = 0
    mask_ctr = 0
    
    for file_name in os.listdir(os.path.join(ROOT, folder, img_folder)):
        
        img = Image.open(os.path.join(ROOT, folder, img_folder, file_name)).convert("RGB")
        
        mask = cv.imread(os.path.join(ROOT, folder, label_folder, file_name), cv.IMREAD_GRAYSCALE) * 255
        mask = Image.fromarray(mask.astype('uint8'), 'L')
    
        base_file_name = file_name.split(".")[0]
        
        if mask.width != 10000 or img.width != 10000 or mask.height != 10000 or img.height != 10000:
            continue
        
        t = transforms.ToTensor()(img.convert("L"))
        x = t[t == 0].numel() / 10000**2
        
        if x > 0.15:
            ctr += 1
            continue
            
        for x0 in range(0, 10000, IMAGE_SIZE):
            for y0 in range(0, 10000, IMAGE_SIZE):
                x1 = x0 + IMAGE_SIZE
                y1 = y0 + IMAGE_SIZE
                
                if x1 > 10000 or y1 > 10000:
                    continue
                
                img_cropped = img.crop((x0, y0, x1, y1))
                mask_cropped = mask.crop((x0, y0, x1, y1))
                
                m = transforms.ToTensor()(mask_cropped)
                
                if m.unique().sum() == 0:
                    mask_ctr += 1
                    continue
                
                img_cropped.save(os.path.join(SAVE_ROOT, IMAGES, f"{PREFIX}_{base_file_name}_{x0}_{y0}.png"), "PNG")
                mask_cropped.save(os.path.join(SAVE_ROOT, LABELS, f"{PREFIX}_{base_file_name}_{x0}_{y0}.png"), "PNG")
                
    print(f"Items not selected: {ctr}")
    print(f"Masks not selected: {mask_ctr}")

test
Items not selected: 4
Masks not selected: 21780
train
Items not selected: 64
Masks not selected: 175830
val
Items not selected: 12
Masks not selected: 18582
